In [ ]:
import random
from datetime import datetime, date, time
from selenium import webdriver
from time import sleep
from functools import partial
import pandas as pd
from glob import glob
from os import path
import re
from hashlib import md5

visited_products = None

def random_sleep(offset=1, length=1):
    sleep(random.random() * length + offset)
    
def wait_for_unblock(browser):
    if browser.title == 'Доступ временно заблокирован':
        print('Have been blocked')
        while browser.title == 'Доступ временно заблокирован':
            sleep(5)
    
def get_visited_products(path=None):
    from os import path as path_
    if path is None:
        path = path_.join('data', 'visited_products.txt')
    try:
        with open(path, 'r') as file:
            return set([product.strip() for product in file])
    except IOError as e:
        print('error in get_visited_products function: ', e)
        return set()
    

def get_visited_accounts(directory='data', formatted='csv'):    
    try:
        urls = set()
        for file_name in glob(path.join(directory, '*.' + formatted)):
            df = pd.read_csv(file_name)
            for long_url in df.url:
                url = re.match(r"https://www.avito.ru/user/(.*)/profile", long_url).group(0)
                urls.add(url)
        return urls
    except:
        print('error in get_visited_accounts function')
        return set()
    
    
def visit_product(url, path=None):
    from os import path as path_
    if path is None:
        path = path_.join('data', 'visited_products.txt')
        
    global visited_products
    try:
        visited_products.add(url)
        with open(path, 'a') as file:
            print(url, file=file)
    except IOError as e:
        print('error in visit_product function: ', e)

        
def get_acc_from_product(driver):
    # 
    long_url = driver.find_element_by_class_name('seller-info-name').find_element_by_tag_name('a').get_attribute('href')
    try:
        url = re.match(r"https://www.avito.ru/user/(.*)/profile", long_url).group(0)
        return url
    except:
        return long_url


def get_accounts_from_page(driver, url):
    #данные о проверенных товарах
    global visited_products
    if visited_products is None:
        visited_products = get_visited_products()

    # Переходим на страничку, откуда будем вытаскивать аккаунты из товаров
    driver.get(url)
    random_sleep(1, 1)
    
    # Если заблочили ip, то выполняем соответствующие действия
    wait_for_unblock(driver)

    # Берём ссылки на все товары и сохраняем в urls
    products = driver.find_elements_by_class_name('item-description-title-link')
    urls = []
    for item in products:
        urls.append(item.get_attribute('href'))
    
    # Ссылки, которые потом вернём
    accounts_list = []
        
    for product_url in urls:
        # Если не чекали этот товар раньше
        if not (product_url in visited_products):
            # Запомнить, что чекнули
            visit_product(product_url)
            
            # Переходим по ссылке на товар
            driver.get(product_url)
            random_sleep(0, 0.3)
            wait_for_unblock(driver)
            
            # И пытаемся вытащить ссылку
            try:
                account_link = get_acc_from_product(driver)
                accounts_list.append(account_link)
            # Не получилось, значит есть за что
            except:
                pass
    
    return accounts_list


def filter_links(urls, accounts, HASH_MOD = [0]):
    '''Функция чекает список новых урлов на акки, отсеивает посещённые, оставляет нужные по хешу и тд и тп'''
    
    result = []
    for url in urls:
        if not (url in accounts) and url.startswith('https://www.avito.ru/user/') and \
                (int(md5(url.encode('utf-8')).hexdigest(), base=16) % 6 in HASH_MOD):
            result.append(url)
    return result


def search(driver, url, HASH_MOD = [2, 3, 4], max_accounts = 0, start_page = 1):
    '''url - ссылка на страничку авито, где есть товары и прокрутка страниц, чтобы можно было добавить
    "?p=<номер страницы> в конец url
    max_accounts - максимальное количество аккаунтов, которое вытаскивает функция search '''
    
    # Вытаскиваем из папки data даные о уже обработанных акках.
    visited_accounts = get_visited_accounts()
    
    # Номер страницы на avito
    page = start_page
    
    # Счётчик возвращённых акков
    yielded_accounts_counter = 0
    
    # ссылки на аккаунты, собранные со странички avito
    links_from_page = []
    
    while True:
        # Выбросили столько, сколко нужно
        if yielded_accounts_counter >= max_accounts:
            break
            
        # Если прошли все акки в links_from_page, то берём новые со следующей страницы.
        while len(links_from_page) == 0:
            new_links = get_accounts_from_page(driver, url=(url + r'?p={}').format(page))
            links_from_page = filter_links(new_links, visited_accounts, HASH_MOD)
            page += 1
            
        account = links_from_page.pop()
        visited_accounts.add(account)
        yielded_accounts_counter += 1
        yield account

        
def parse(browser, url):
    browser.get(url)
    wait_for_unblock(browser)
    
    name = browser.find_element_by_class_name('profile-public-name-edUBF')
    data = browser.find_elements_by_class_name('profile-public-summary-item-wFbyw')
    prod = browser.find_elements_by_class_name('styles-date-2vN1V')
    cost = browser.find_elements_by_class_name('styles-price-1RC3V')
    i = 0
    l = len(prod) - 1
    l2 = l
    l3 = l2
    while(l != len(prod)):
        l3 = l2
        l = len(prod)
        l2 = l
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        random_sleep()
        prod = browser.find_elements_by_class_name('styles-date-2vN1V')
        cost = browser.find_elements_by_class_name('styles-price-1RC3V')
        try:
            element = browser.find_element_by_class_name('styles-button-10MP4')
            element.click()
            l = l - 1
            random_sleep()
        except:
            pass
        if(l3 == len(prod)):
            l = len(prod)
    person_info = {
        'url': url,
        'name': name.text,
        'data': list(map(lambda x: x.text, data)),
        'prod_data_active': list(map(lambda x: x.text, prod)),
        'prod_cost_active': list(map(lambda x: x.text, cost))
    }
    c = browser.find_element_by_partial_link_text('Завершённые')
    c.click()
    random_sleep()
    prodold = browser.find_elements_by_class_name('styles-date-2vN1V')
    costold = browser.find_elements_by_class_name('styles-price-1RC3V')
    l = len(prodold) - 1
    l2 = l
    l3 = l2
    if browser.page_source.find('Нет завершённых объявлений') < 0:
        while(l != len(prodold)):
            l3 = l2
            l = len(prodold)
            l2 = l
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            random_sleep()
            prodold = browser.find_elements_by_class_name('styles-date-2vN1V')
            costold = browser.find_elements_by_class_name('styles-price-1RC3V')
            try:
                element = browser.find_element_by_class_name('styles-button-10MP4')
                element.click()
                l = l - 1
                random_sleep()
            except:
                pass
            if(l3 == len(prod)):
                l = len(prodold)

    person_info['prod_data_finished'] = list(map(lambda x: x.text, prodold))
    person_info['prod_cost_finished'] = list(map(lambda x: x.text, costold))
    return person_info


def strtodays(s):
    months = ['января', 'февраля', 'марта', 'апреля', 'мая', 'июня',
                  'июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря'] 
    
    if s.lower().find('сегодня') >= 0:
        t = date.today()
        return date(t.year, t.month, t.day).toordinal()
    elif s.lower().find('вчера') >= 0:
        t = date.today() - timedelta(days=1)
        return date(t.year, t.month, t.day).toordinal()
    
    m = re.match(r'.*(\d\d\d\d).*', s)
    year = date.today().year if m is None else int(m.group(1))
    month = 0
    for i in range(12):
        if s.find(months[i]) >= 0:
            month = i + 1
            break
    day = int(re.match(r'(\d*) .*', s).groups(1)[0])

    return date(year, month, day).toordinal()


def normalize(account):
    result = dict()
    result['name'] = account['name'].lower()
    result['person_status'] = account['data'][0].lower()
    result['url'] = account['url']
    
    start_date = account['data'][1]
    if start_date.startswith('На Авито с '):
        months = ['января', 'февраля', 'марта', 'апреля', 'мая', 'июня',
                  'июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря'] 
        month = 0
        for i in range(12):
            if start_date.find(months[i]) >= 0:
                month = i + 1
                break
        result['start_date'] = str(date(int(re.match('.*(\d\d\d\d).*', start_date).group(1)), month, 1).toordinal())
    else:
        result['start_date'] = ''
        
    result['answer_time'] = account['data'][3]
    result['first_sold_time'] = '' if len(account['prod_data_finished']) == 0 else str(strtodays(account['prod_data_finished'][-1]))
    result['n_active'] = str(len(account['prod_data_active']))
    result['n_finished'] = str(len(account['prod_data_finished']))
    result['active_finished_ratio'] = str((len(account['prod_data_active']) + 1) / (len(account['prod_data_finished']) + 1))
    
    active_costs = []
    for cost in account['prod_cost_active']:
        active_costs.append(int(re.match('(\d*).*', cost.replace(' ', '')).group(1)))
        
    finished_costs = []
    for cost in account['prod_cost_finished']:
        finished_costs.append(int(re.match('(\d*).*', cost.replace(' ', '')).group(1)))
        
    result['min_active_cost'] = str(min(active_costs)) if len(active_costs) > 0 else ''
    result['max_active_cost'] = str(max(active_costs)) if len(active_costs) > 0 else ''
    result['mean_active_cost'] = str(sum(active_costs) / len(active_costs)) if len(active_costs) > 0 else ''
    
    result['min_finished_cost'] = str(min(finished_costs)) if len(finished_costs) > 0 else ''
    result['max_finished_cost'] = str(max(finished_costs)) if len(finished_costs) > 0 else ''
    result['mean_finished_cost'] = str(sum(finished_costs) / len(finished_costs)) if len(finished_costs) > 0 else ''
    return result

if __name__ == '__main__':
    max_accounts = 3000
    url = r'https://www.avito.ru/rossiya/lichnye_veschi'
    data = {}
    browser = webdriver.Chrome()
    parser = partial(parse, browser)
    searcher = search(driver=browser, url=url, max_accounts=max_accounts)
    for key, item in normalize(parser(next(searcher))).items():
        data[key] = [item]
    counter = 0
    cntr = 0
    while(cntr < max_accounts):
        try:
            for acc_data in map(parser, searcher):
                if(counter == 15):
                    df.to_csv(path.join('data', 'avito_data2.csv'))
                counter += 1
                counter %= 30
                cntr += 1
                for key, item in normalize(acc_data).items():
                    data[key].append(item)
                df = pd.DataFrame(data)
                if(counter == 0):
                    df.to_csv(path.join('data', r'avito_data{}.csv').format(str(datetime.now()).replace(':', '.')))
                    for key in data.keys():
                        data[key] = []
                print(cntr)
        except:
            df.to_csv(path.join('data', r'avito_data{}.csv').format(str(datetime.now()).replace(':', '.')))
            for key in data.keys():
                data[key] = []
            random_sleep(60, 60)
            cntr += 1


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
Have been blocked
